In [1]:
%matplotlib inline
import os
import cv2
from fastai.learner import load_learner
from fastai.vision.core import PILImage
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import shutil
import mediapipe as mp
import math
import tempfile

def get_pitcher_handedness(file_path):
    return 'left_handed' if 'left_handed' in str(file_path) else 'right_handed'

def get_open_side(file_path):
    file_path_parts = str(file_path).split(os.sep)
    return 'open_side' if 'open_side' in file_path_parts else 'not_open_side'

# Load the three models
pitching_model = load_learner('pitching_model.pkl')
left_open_side_model = load_learner('left_open_side_model.pkl')
right_open_side_model = load_learner('right_open_side_model.pkl')

In [2]:
# Define your list of video files
videos = ['C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video1.avi','C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video2.avi','C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video3.avi','C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video4.avi','C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video5.avi','C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video6.avi','C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video7.avi','C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video8.avi']

In [3]:
def clear_output_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
    os.makedirs(folder_path, exist_ok=True)

def extract_frames_from_videos(videos, output_folder, n_frames=10):
    for video_id, video_path in enumerate(videos):
        video_output_folder = os.path.join(output_folder, f'video_{video_id}')
        
        print(f"Processing video: {video_path}")
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Unable to open video file {video_path}")
            continue

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = max(1, frame_count // n_frames)

        for i in range(0, frame_count, step):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break
            
            os.makedirs(video_output_folder, exist_ok=True)
            output_path = os.path.join(video_output_folder, f'frame_{i:04d}.png')
            cv2.imwrite(output_path, frame)

        cap.release()
        cv2.destroyAllWindows()

def display_extracted_frames(output_folder, n_videos):
    for video_id in range(n_videos):
        video_output_folder = os.path.join(output_folder, f'video_{video_id}')
        if os.path.exists(video_output_folder):
            files = os.listdir(video_output_folder)
            for file in files:
                img_path = os.path.join(video_output_folder, file)
                frame = cv2.imread(img_path)
                if frame is not None:
                    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    plt.title(f'{img_path}')
                    plt.axis('off')
                    plt.show()

output_folder = 'combined_extracted_frames'
clear_output_folder(output_folder)
extract_frames_from_videos(videos, output_folder, n_frames=10)

Processing video: C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video1.avi
Processing video: C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video2.avi
Processing video: C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video3.avi
Processing video: C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video4.avi
Processing video: C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video5.avi
Processing video: C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video6.avi
Processing video: C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video7.avi
Processing video: C:/Users/OSUsp/Desktop/AVI Trimming Model/test/video8.avi


In [4]:
handiness_results = []
n_frames = 10

for video_id in range(len(videos)):
    video_output_folder = os.path.join(output_folder, f'video_{video_id}')
    print(f'Processing video {video_id} in folder: {video_output_folder}')

    if os.path.exists(video_output_folder):
        files = os.listdir(video_output_folder)
        step = max(1, len(files) // n_frames)
        predictions = []
        frames = []

        # Load extracted frames into a list
        for file in files[::step]:
            img_path = os.path.join(video_output_folder, file)
            img = PILImage.create(img_path)
            frames.append(img)

        # Predict handiness for each frame
        for frame in frames:
            pred_class, _, _ = pitching_model.predict(frame)
            predictions.append(str(pred_class))

        if not predictions:
            handiness = "unknown"
        else:
            handiness, _ = Counter(predictions).most_common(1)[0]
    else:
        print(f'Folder not found: {video_output_folder}')
        handiness = "unknown"

    handiness_results.append(handiness)
    print(f'Video {video_id} Handiness: {handiness}')



Processing video 0 in folder: combined_extracted_frames\video_0


Video 0 Handiness: right_handed
Processing video 1 in folder: combined_extracted_frames\video_1


Video 1 Handiness: right_handed
Processing video 2 in folder: combined_extracted_frames\video_2


Video 2 Handiness: right_handed
Processing video 3 in folder: combined_extracted_frames\video_3


Video 3 Handiness: right_handed
Processing video 4 in folder: combined_extracted_frames\video_4


Video 4 Handiness: right_handed
Processing video 5 in folder: combined_extracted_frames\video_5


Video 5 Handiness: right_handed
Processing video 6 in folder: combined_extracted_frames\video_6


Video 6 Handiness: right_handed
Processing video 7 in folder: combined_extracted_frames\video_7


Video 7 Handiness: right_handed


In [5]:
def predict_open_side_on_frames(output_folder, model, n_frames, n_videos):
    all_video_results = []

    for video_id in range(n_videos):
        video_output_folder = os.path.join(output_folder, f'video_{video_id}')
        video_results = []

        for i in range(n_frames):
            img_path = os.path.join(video_output_folder, f'frame_{i:04d}.png')
            if os.path.exists(img_path):
                img = PILImage.create(img_path)
                pred, _, _ = model.predict(img)
                video_results.append(pred)
                print(f'Video {video_id} Frame {i:04d}: {pred}')  # Debugging line

        if video_results:
            most_common_prediction, _ = Counter(video_results).most_common(1)[0]
            all_video_results.append((video_id, most_common_prediction))

    return all_video_results


n_frames = 10 

handiness_counter = Counter(handiness_results)
handiness, _ = handiness_counter.most_common(1)[0]
print(f'Pitcher Hand: {handiness}')

if handiness == 'left_handed':
    model = left_open_side_model
else:
    model = right_open_side_model

open_side_results = predict_open_side_on_frames(output_folder, model, n_frames, len(videos))

for video_id, open_side in open_side_results:
    print(f'Video {video_id}: Open Side: {open_side}')

Pitcher Hand: right_handed


Video 0 Frame 0000: not_open_side


Video 1 Frame 0000: not_open_side


Video 2 Frame 0000: not_open_side


Video 3 Frame 0000: not_open_side


Video 4 Frame 0000: not_open_side


Video 5 Frame 0000: open_side


Video 6 Frame 0000: not_open_side


Video 7 Frame 0000: not_open_side
Video 0: Open Side: not_open_side
Video 1: Open Side: not_open_side
Video 2: Open Side: not_open_side
Video 3: Open Side: not_open_side
Video 4: Open Side: not_open_side
Video 5: Open Side: open_side
Video 6: Open Side: not_open_side
Video 7: Open Side: not_open_side


In [6]:
# Calculate max knee height function
def calculate_max_knee_height(video_path, handiness):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, model_complexity=2)

    max_knee_height = 0
    max_knee_height_frame_index = -1
    frame_index = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmark = results.pose_landmarks.landmark

            if handiness == 'right_handed':
                hip = mp_pose.PoseLandmark.LEFT_HIP
                knee = mp_pose.PoseLandmark.LEFT_KNEE
            else:
                hip = mp_pose.PoseLandmark.RIGHT_HIP
                knee = mp_pose.PoseLandmark.RIGHT_KNEE

            hip_y = landmark[hip.value].y
            knee_y = landmark[knee.value].y

            knee_height = hip_y - knee_y
            if knee_height > max_knee_height:
                max_knee_height = knee_height
                max_knee_height_frame_index = frame_index

        frame_index += 1

    cap.release()
    cv2.destroyAllWindows()

    return max_knee_height_frame_index, fps

def calculate_start_frame_index(max_knee_height_frame_index, fps, ms_before=4000):
    start_frame_index = max_knee_height_frame_index - math.ceil((ms_before / 1000) * fps)
    return max(0, start_frame_index)

def trim_video(video_path, start_frame_index, output_path):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index >= start_frame_index:
            out.write(frame)

        frame_index += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Find the index of the open side video
open_side_video_index = None
for index, (video_id, open_side) in enumerate(open_side_results):
    if open_side == 'open_side':
        open_side_video_index = index
        break

if open_side_video_index is not None:
    # Calculate max knee height for open side video
    open_side_video_path = videos[open_side_video_index]
    open_side_handiness = handiness_results[open_side_video_index]
    max_knee_height_frame_index, fps = calculate_max_knee_height(open_side_video_path, open_side_handiness)
    start_frame_index = calculate_start_frame_index(max_knee_height_frame_index, fps)

    # Trim all videos based on the max knee height calculation
    for video_path in videos:
        video_basename = os.path.basename(video_path)
        video_filename, video_extension = os.path.splitext(video_basename)
        trimmed_video_path = os.path.join(os.path.dirname(video_path), f"{video_filename}_trimmed{video_extension}")

        trim_video(video_path, start_frame_index, trimmed_video_path)
        print(f'Trimmed video saved to: {trimmed_video_path}')

        # Uncomment the line below to overwrite the original videos with the trimmed versions
        shutil.move(trimmed_video_path, video_path)

else:
    print('Open side video not found in the list of videos.')



Trimmed video saved to: C:/Users/OSUsp/Desktop/AVI Trimming Model/test\video1_trimmed.avi
Trimmed video saved to: C:/Users/OSUsp/Desktop/AVI Trimming Model/test\video2_trimmed.avi
Trimmed video saved to: C:/Users/OSUsp/Desktop/AVI Trimming Model/test\video3_trimmed.avi
Trimmed video saved to: C:/Users/OSUsp/Desktop/AVI Trimming Model/test\video4_trimmed.avi
Trimmed video saved to: C:/Users/OSUsp/Desktop/AVI Trimming Model/test\video5_trimmed.avi
Trimmed video saved to: C:/Users/OSUsp/Desktop/AVI Trimming Model/test\video6_trimmed.avi
Trimmed video saved to: C:/Users/OSUsp/Desktop/AVI Trimming Model/test\video7_trimmed.avi
Trimmed video saved to: C:/Users/OSUsp/Desktop/AVI Trimming Model/test\video8_trimmed.avi
